In [21]:
# coding: utf-8
import pandas as pd
import jieba
import paddle
import time
import csv
from collections import Counter

In [2]:
#---------------------------------------中文分词---------------------------------
#添加使用者词典和停用词
paddle.enable_static()
jieba.enable_paddle()
jieba.load_userdict("../resource/dict.txt.big")              #自定义词典
stopwords = [line.strip() for line in open('../resource/stopwords.txt', 'r', encoding='utf-8').readlines()]

def txt_cut(juzi):
    return [w for w in jieba.lcut(juzi) if w not in stopwords]     #可增加len(w)>1

Paddle enabled successfully......
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/q9/0yjtkz2x1mq4f7qlvql797l80000gn/T/jieba.cache
Loading model cost 1.323 seconds.
Prefix dict has been built successfully.


In [3]:
df = pd.read_excel('../resource/情感词汇.xlsx')
df = df[['词语', '词性种类', '词义数', '词义序号', '情感分类', '强度', '极性']]

In [4]:
#-------------------------------------七种情绪的运用-------------------------------
Happy = []
Good = []
Surprise = []
Anger = []
Sad = []
Fear = []
Disgust = []

#df.iterrows()功能是迭代遍历每一行
for idx, row in df.iterrows():
    if row['情感分类'] in ['PA', 'PE']:
        Happy.append(row['词语'])
    if row['情感分类'] in ['PD', 'PH', 'PG', 'PB', 'PK']:
        Good.append(row['词语']) 
    if row['情感分类'] in ['PC']:
        Surprise.append(row['词语'])       
    if row['情感分类'] in ['NB', 'NJ', 'NH', 'PF']:
        Sad.append(row['词语'])
    if row['情感分类'] in ['NI', 'NC', 'NG']:
        Fear.append(row['词语'])
    if row['情感分类'] in ['NE', 'ND', 'NN', 'NK', 'NL']:
        Disgust.append(row['词语'])
    if row['情感分类'] in ['NAU']:     #修改: 原NA算出来没结果
        Anger.append(row['词语'])  

#正负计算不是很准 自己可以制定规则       
Positive = Happy + Good + Surprise
Negative = Anger + Sad + Fear + Disgust

In [5]:
text = open('../1.txt',encoding='utf8').read()

In [6]:
#---------------------------------------情感计算---------------------------------
#文件写入
c = open("Emotion_features.csv", "a+", newline='', encoding='gb18030')
writer = csv.writer(c)
writer.writerow(["Emotion","Word","Num"])

18

In [10]:
#情感统计
def emotion_caculate(text):
    positive = 0
    negative = 0
    
    anger = 0
    disgust = 0
    fear = 0
    sad = 0
    surprise = 0
    good = 0
    happy = 0

    anger_list = []
    disgust_list = []
    fear_list = []
    sad_list = []
    surprise_list = []
    good_list = []
    happy_list = []
    
    wordlist = txt_cut(text)
    #wordlist = jieba.lcut(text)
    wordset = set(wordlist)
    wordfreq = []
    for word in wordset:
        freq = wordlist.count(word)
        tlist = []
        if word in Positive:
            positive+=freq
        if word in Negative:
            negative+=freq
        if word in Anger:
            anger+=freq
            anger_list.append(word)
            tlist.append("anger")
            tlist.append(word)
            tlist.append(freq)
            writer.writerow(tlist)
        if word in Disgust:
            disgust+=freq
            disgust_list.append(word)
            tlist.append("disgust")
            tlist.append(word)
            tlist.append(freq)
            writer.writerow(tlist)
        if word in Fear:
            fear+=freq
            fear_list.append(word)
            tlist.append("fear")
            tlist.append(word)
            tlist.append(freq)
            writer.writerow(tlist)
        if word in Sad:
            sad+=freq
            sad_list.append(word)
            tlist.append("sad")
            tlist.append(word)
            tlist.append(freq)
            writer.writerow(tlist)
        if word in Surprise:
            surprise+=freq
            surprise_list.append(word)
            tlist.append("surprise")
            tlist.append(word)
            tlist.append(freq)
            writer.writerow(tlist)
        if word in Good:
            good+=freq
            good_list.append(word)
            tlist.append("good")
            tlist.append(word)
            tlist.append(freq)
            writer.writerow(tlist)
        if word in Happy:
            happy+=freq
            happy_list.append(word)
            tlist.append("happy")
            tlist.append(word)
            tlist.append(freq)
            writer.writerow(tlist)
            
    emotion_info = {
        'length':len(wordlist),
        'positive': positive,
        'negative': negative,
        'anger': anger,
        'disgust': disgust,
        'fear':fear,
        'good':good,
        'sadness':sad,
        'surprise':surprise,
        'happy':happy,
        
    }

    indexs = ['length', 'positive', 'negative', 'anger', 'disgust','fear','sadness','surprise', 'good', 'happy']
    return pd.Series(emotion_info, index=indexs), anger_list, disgust_list, fear_list, sad_list, surprise_list, good_list, happy_list

In [14]:
#---------------------------------------情感计算--------------------------------- 
emotion_df = emotion_caculate(text)

In [25]:
from pyecharts import options as opts
from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType

# 渲染图
def wordcloud_base() -> WordCloud:
    c = (
        WordCloud()
        .add("", result, word_size_range=[5, 200], shape=SymbolType.ROUND_RECT)
        .set_global_opts(title_opts=opts.TitleOpts(title='情绪词云图'))
    )
    return c


# 生成图
#wordcloud_base().render('情绪词云图.html')



In [31]:
for i in range(1,8):
    total_terms=sorted(Counter(emotion_df[i]).items(), key=lambda x:x[1], reverse=True)
    def wordcloud_base() -> WordCloud:
        c = (
            WordCloud()
            .add("", total_terms, word_size_range=[5, 200], shape=SymbolType.ROUND_RECT)
            .set_global_opts(title_opts=opts.TitleOpts(title='情绪词云图'))
        )
        return c
    wordcloud_base().render('{}.html'.format(i))

In [34]:
total_terms=sorted(Counter(emotion_df[2]).items(), key=lambda x:x[1], reverse=True)
total_terms

[('蓄意', 1),
 ('威胁', 1),
 ('大胆', 1),
 ('牵扯', 1),
 ('变态', 1),
 ('犯罪', 1),
 ('好人', 1),
 ('软弱', 1),
 ('野心', 1),
 ('胡扯', 1),
 ('损害', 1),
 ('差劲', 1),
 ('不良', 1),
 ('无情', 1),
 ('愚蠢', 1),
 ('手段', 1),
 ('该死', 1),
 ('抛弃', 1),
 ('抢劫', 1),
 ('开心', 1),
 ('骗人', 1),
 ('对付', 1),
 ('黑市', 1),
 ('王八', 1),
 ('糟糕', 1),
 ('棘手', 1),
 ('婊子', 1),
 ('误会', 1),
 ('胆小', 1),
 ('记恨', 1),
 ('驳回', 1),
 ('打扰', 1),
 ('欺瞒', 1),
 ('解决', 1),
 ('介意', 1),
 ('失禁', 1),
 ('暴力', 1),
 ('囚犯', 1),
 ('忍受', 1),
 ('虚伪', 1),
 ('私利', 1),
 ('权势', 1),
 ('夸大', 1),
 ('担心', 1),
 ('浪费', 1),
 ('糊涂', 1),
 ('笑话', 1),
 ('剥削', 1),
 ('诈骗', 1),
 ('白痴', 1),
 ('欺骗', 1),
 ('无耻', 1),
 ('生气', 1),
 ('积怨', 1),
 ('受苦', 1),
 ('畏首畏尾', 1),
 ('随便', 1),
 ('蠢货', 1),
 ('混蛋', 1),
 ('贱人', 1),
 ('绕圈子', 1),
 ('盘根错节', 1),
 ('冷落', 1),
 ('失败', 1),
 ('牟取', 1),
 ('憋屈', 1),
 ('窃取', 1),
 ('收买', 1),
 ('损失', 1),
 ('逃跑', 1),
 ('残暴', 1),
 ('谋杀', 1),
 ('谢绝', 1),
 ('繁重', 1),
 ('串通', 1),
 ('强奸', 1),
 ('麻烦', 1),
 ('虚构', 1),
 ('轻易', 1),
 ('违背', 1),
 ('毫无意义', 1),
 ('坏人', 1),
 ('伪造', 1